## Assignment: Create a Chatbot

### Name: Nikiforos Mandilaras

### Email: nikiforosmandi@windowslive.com

### Date: 15/11/2019

### Introduction

This Notebook was developed as an assignment for the needs of National Center of Scientific Research Demokritos.
The goal is to create a chatbot, namely a conversatonal model that can give responses to the user's inputs. 

### Dependency Packaging & Execution

In order to be able to execute this jupyter notebook, all depedent packages must be present. For this reason we save those along with their versions in requirements.txt (which is included in the submission folder). 

Then we can install them all at once using pip as it can be seen below. To avoid any collisions with existing packages it is necessary to create a new python virtual enviroment and run the command inside there. 

To access data the following code uses relative paths, assuming data are present in the same folder as the jupyter notebook. However all functions can accept absolute paths as well.  

In [54]:
# uncomment to install dependencies

# !pip install -r requirements.txt

### Approach

To address this task we are going to need a pretrained language model.
Many such state of the art models are included in the widely used library [trasformers](https://github.com/huggingface/transformers) [1] developped by **_Hugging Face_**. We choose to use GPT as it is pretrained on predicting the next word and compared to other models it's relatively small since our resources are limited. 

More details about the approach are provided in the report.

In [1]:
import os
import pickle
import numpy as np
import glob
import copy
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader 
from datetime import datetime
from transformers import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, WEIGHTS_NAME, CONFIG_NAME
from itertools import chain
from ast import literal_eval
from itertools import zip_longest
from sklearn.model_selection import train_test_split

### Initializing Model

Below we instantiate a GPT Pytorch model with pre-trained weights on language modelling task along with tokenizer.
Tokenizer is a helper class used to interact with the vocabulary in which our model has been pretrained.

In [7]:
def load_checkpoint(output_dir='openai-gpt'):
    """
    Loads GPT Model and the corresponding tokenizer from local checkpoint.
    If no path is specified the pretrained weights on language modelling task are downloaded.
    
    :param output_dir: path to checkpoint 
    :return : model and tokenizer
    """

    tokenizer = OpenAIGPTTokenizer.from_pretrained(output_dir)
    model = OpenAIGPTLMHeadModel.from_pretrained(output_dir)  
    
    return model, tokenizer

In [38]:
model, tokenizer = load_checkpoint() 

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [4]:
print("Our language model have been pre-trained with a vocabulary of {} words.".format(tokenizer.vocab_size))

Our language model have been pre-trained with a vocabulary of 40478 words.


In [5]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of trainable model parameters: {}".format(trainable_params))

Number of trainable model parameters: 116534784


As we can see GPT model has been loaded and it consists of 12 layers with a total of more than 116 millios of trainable parameters. We avoided using the more recent GPT-2 as it has 1.5 billions parameters and that may causes issues loading it into RAM. 

In [9]:
def add_special_tokens_(model, tokenizer, special_tokens):
    """
    Adds special tokens to tokenizer and model.
    """
    orig_num_tokens = len(tokenizer.encoder)
    num_added_tokens = tokenizer.add_special_tokens(special_tokens) # adds only if not present
    
    if num_added_tokens > 0:
        print("New tokens added to model: {}".format(num_added_tokens))
        model.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens) 
    else:
        print("No new tokens found! Nothing added.")

We saw earlier that the pretrained model uses a vocabulary of around forty thousands words. Apart from those for our task we will need some extra tokens with special meaning. Hopefully tokenizer can handle this process for us.

In [22]:
# various constants needed

SPECIAL_TOKENS_DICT = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ('<speaker1>', '<speaker2>')}

max_history = 2 # pairs of question/answer to be retained
max_sentence_length = 20 # maximum length of a sentence produced by the model 

temperature = 0.75 # increases confidence in the most propable outputs 

use_cuda = True # whether to try to use cuda or not

SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
add_special_tokens_(model, tokenizer, SPECIAL_TOKENS_DICT)  

SPECIAL_TOKENS_IDS = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
TIME_FORMAT = '%Y%m%d_%H%M'

We added 5 special tokens for denoting the start and the end of the sentences, the type of the input provided to the model(whether it came from the user or the bot) and a finally a token to denote padding. 

### Data Preprocessing

In this section we define a series of functions to appropiate handle the data. At first with function **_parser_** we read the txt files and we keep only the dialogs in a list. Also we tokenize phrases and we convert all words to their corresponding ids in the model's vocabulary. This step is performed early before we create multiple copies of each sentence in many samples as part of history, to optimize performance. 

In [27]:
def parser(datafolder='metalwoz-v1/dialogues/', cache_file='cache_folder/dialogs.txt'): 
    """
    Function that reads files, keeps only 'turns' from each entry and tokenizes them

    :param datafolder: path to the folder that contains the files
    :param cache_file: filepath to save the result
    :return: a list that contains dialogs, each dialog is a list of lists 
             where each of them represents the ids of a phrase,
    """
    try:    # try to open cache file
        with open(cache_file, "rb") as f:
            print("Cache file found loading content.")
            dialogs = pickle.load(f)
            return dialogs
        
    except: # cache file not created yet
        print("Cache file not found. Start processing.")       
    
        dialogs = []
        files = list(glob.glob(os.path.join(datafolder ,"*.txt")))

        for file in files:
            with open(file) as f:
                for line in f.readlines():

                    dialog = literal_eval(line)['turns'][1:] # keep only turns without the first sentence
                    dialog = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(phrase)) for phrase in dialog]  
                    dialogs.append(dialog) 

        if len(dialogs) > 0:
            print("Saving parsed dialogs to file: {}".format(cache_file))
            with open(cache_file, "wb") as f: # save result so future calls can retrieve it right away
                pickle.dump(dialogs, f)            
                
    return dialogs      

Then with the function **_extract_pairs_** we use those dialogs to create utterances of input, output pairs. In every such pair we preserve in the input the entire past history of the dialog. Additionally we discard the first sentence , which is fixed, and the last one when it said by the user, as no bot answer follows. 

In [28]:
def extract_pairs(dialogs = None, cache_file='cache_folder/pairs.txt'):
    """
    Function that creates pairs of input, output from dialogs, each dialogs corresponds now to many pairs.
    
    :param dialogs: a list with all the dialogs 
    :param cache_file: filepath to save the result
    
    :return : a list whose elements are pairs of input(history), output(expected bot reply)  
    """
    try:    # try to open cache file
        
        with open(cache_file, "rb") as f:
            print("Cache file found loading content.")
            pairs = pickle.load(f)       
            return pairs
        
    except:      # cache file not created yet
        print("Cache file not found. Start processing.")
        
        pairs = [] 
        for dialog in dialogs:
            
            t_dict = {'input': []}
            if len(dialog) % 2 != 0: # discard the last phrase if it was said by the user
                dialog = dialog[:-1]    
            dialog_it = iter(dialog)
            
            for i_phrase, o_phrase in zip_longest(dialog_it, dialog_it): # process phrases two by two        
                try:
                    t_dict["input"].append(t_dict["output"])
                except:
                    pass 
                t_dict["input"].append(i_phrase) # history
                t_dict["output"] = o_phrase
                pairs.append(t_dict)
                t_dict = copy.deepcopy(t_dict) # so future changes address only the new dict
           
        if len(pairs) > 0:
            print("Saving extracted pairs to file: {}".format(cache_file))          
            with open(cache_file, "wb") as f:  # save result so future calls can retrieve it right away
                pickle.dump(pairs, f)
            
        return pairs

A couple of auxilary functions follows. At first, **_adjust_history_** helps us to tune the number of quenstion/answer pairs retained in history. Obviously in every case at least one sentence from the user is kept as input. We conducted training experiments for different values of retained history in order to observe if and it what extend it affects model's performance.

In [29]:
def adjust_history(pairs, max_history=2): 
    """
    Reduces number of previous chat senteces that are going to be included in the input
    
    :param pairs: list with samples 
    :param max_history: Number of pairs of question/answer to be preserved (at least one is preserved)
    :return : two lists, pairs with fixed history and theirs corresponding seq_lenghts 
    """
    
    pairs_len = []
    for pair in pairs:
        
        pair['input'] = pair['input'][-(2*max_history+1):] # at least one phrase is preserved
        pair_len = sum(len(phrase) for phrase in pair['input']) + len(pair['output'])
        pairs_len.append(pair_len)
        
    return pairs, pairs_len   

Secondly, **_filter_samples_** assesses the samples based on their sequense length. Samples that their length exceeds a specified percentile are dropped. The impact of this function is crusial in terms of performance as we will see later. 

In [30]:
def filter_samples(samples, samples_len, percentile=90):
    """
    Filters samples based on sequence lengths.  
    
    :param samples: a list with samples
    :param samples_len: their corresponding lengths
    :param percentile: percentage of samples to preserve  
    
    :return : two lists, preserved samples and their lengths
    """
    
    samples_length = np.array(samples_len)
    reasonable_length = np.percentile(samples_length, percentile)
    print("{}% of the samples have sequence length less than {}".format(percentile, reasonable_length))
    
    samples_red, samples_len_red = [], []
    for sample, sample_len in zip(samples, samples_len):
        
        if sample_len <= reasonable_length:
            samples_red.append(sample)
            samples_len_red.append(sample_len)
    
    return samples_red, samples_len_red  

Executing all the aforementioned procedures leads to the following results.

In [31]:
dialogs = parser()

print("Number of dialogs in the whole dataset: {}".format(len(dialogs)))

pairs = extract_pairs(dialogs) # list of dictionaries of input history and bot's reply

# keep only portion of the chat history to reduce seq_length
pairs, pairs_len = adjust_history(pairs, max_history=max_history) 

print("Number of pairs created: {}".format(len(pairs)))
print("Maximum seq_length observed: {}".format(max(pairs_len)))

pairs_reduced, pairs_len_reduced = filter_samples(pairs, pairs_len) 
print("Number of pairs retained: {}".format(len(pairs_reduced)))

Cache file found loading content.
Number of dialogs in the whole dataset: 37884
Cache file found loading content.
Number of pairs created: 193985
Maximum seq_length observed: 684
90% of the samples have sequence length less than 79.0
Number of pairs retained: 175289


We can see that almost 200000 samples were created with a max sequence length of 684 tokens. This is a huge problems for training as every batch that comes into the model is a tensor with fixed dimensions. So the shape of this tensor is decided by the sample with the maximum length on the batch(smaller samples are padded). As a result training times increase dramatically.

Using **_filter_samples_** to cut off the largest 10% of the samples reduces the sequnce length to 79, which something that we can handle in compination with other techniques used afterwards.

### Shaping Data for training

For the needs of training procedure we split our dataset to train and validation set. Validation set is used to monitor the performance of the model at every training epoch.  

In [32]:
pairs_train, pairs_eval = train_test_split(pairs_reduced, test_size=0.3, shuffle=True)      

print("Number of samples in train set: {} and in validation: {}".format(len(pairs_train), len(pairs_eval)))

Number of samples in train set: 122702 and in validation: 52587


The input to our model consists of 4 components. Firstly the history along with the expected reply are given to the model. Apart from that the type(whether each token came from the user or the bot) is provided. 

The labels, namely the tokens of the reply are provided again separately to the model. The labels are shifted inside the model by one position so that outputs that are in position before the n token are trained to predict the token at position n. The output of the model is a 3D tensor with shape (batch_size, sequence_length, vocabulary_size). **_CrossEntropyLoss_** is calculated between this tensor and provided labels. 

Final segment is the attention mask which denotes where real values ends, to prevent attention mechanism for taking into consideration padding. 

Insiration about the inputs of the model was taken by [this](https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313) medium tutorial [2]. 

Given the specified arguments the above procedure is calculated by the function **_create_model_inputs_**.

In [33]:
def create_model_inputs(history, reply, tokenizer, with_eos=True):
    """
    Function that creates the various parts of the model input from input/output pairs.
    """
    
    bos, eos, speaker1, speaker2 = SPECIAL_TOKENS_IDS[:-1]
    sequence = [[bos]] + history + [reply + ([eos] if with_eos else [])]
    seq_len = len(sequence) # sequence: list of lists
    sequence = [sequence[0]] + [[speaker2 if (seq_len-i) % 2 != 1 else speaker1] + s 
                                for i, s in enumerate(sequence[1:])]
    
    instance = {}
    instance["input_ids"] = list(chain(*sequence)) # words
    instance["token_type_ids"] = [speaker1] + [speaker2 if i % 2 else speaker1 
                                               for i, s in enumerate(sequence[1:]) for _ in s] # for each word
    instance["mask"] = [1] * len(instance["input_ids"])                                     # attention mask
    instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
    # TODO positional embeddings
    
    return instance

In the next blocks we use Dataset and Dataloader classes to help us create batches from our data. DialogDataset class wraps the data and it takes care of creating input segments from samples as well as it sorts samples to optimize padding.

In [34]:
class DialogDataset(Dataset):
    """
    Class that wraps data. It takes care of creating input segments from samples as well as
    it sorts samples to optimize padding. Used by the dataloader to sample entries.
    """

    def __init__(self, dialog_pairs):
        self.dataset = self.create_segments(dialog_pairs) # create the segments of the input
        self.dataset = self.sort_on_seq_length() # sorting

    def __len__(self):
        return len(self.dataset)
    
    def create_segments(self, dialog_pairs):
        """
        Creates input segments for each sample
        """
        dataset = []
        for pair in dialog_pairs:
            instance = create_model_inputs(pair['input'], pair['output'], tokenizer)
            dataset.append(instance)
        return dataset
    
    def sort_on_seq_length(self): # could be optimized to use bucket sorting as the number of sample is big
        """
        Sorts dataset based on seq_len to minimize padding afterwards
        """
        return sorted(self.dataset, key=lambda x: len(x['input_ids']))
    

    def __getitem__(self, index):
        return  self.dataset[index]

Dataloader has the role of getting samples from the specified Dataset object and collate them together to form batches. We want to avoid padding the dataset in the global level as by doing so every batch will be of the maximum sequence length. Instead we sorted all samples insided dataset class based on their length and we are preventing Dataloader to sample them at random so for this order to be preserved. 

In this way samples of similar size end up in the same batch and padding is performed inside **_custom_collate_fn_** on a batch level. 

In [39]:
def pad_sequenses(batch, pad_token=0):
    """
    Pads a list of tokens. Padding token differentiates for each input segment 
    """
    max_seq_len = max(len(entry["input_ids"]) for entry in batch)
    
    for entry in batch:
        for index_name in entry.keys():
            
            if index_name == "lm_labels":
                pad_token_ = -1
            elif index_name == "mask":
                pad_token_ = 0
            else:
                pad_token_ = pad_token
                
            entry[index_name] =  entry[index_name] + [pad_token_] * (max_seq_len - len(entry[index_name]))
  
    return batch  

In [40]:
def custom_collate_fn(batch):
    """
    Function that is provided by samples and stacks them together to form tensors.
    We call padding function here. 
    
    :param batch: a list of sambles
    :reutrn : input segments as tensors
    """
    batch = pad_sequenses(batch, SPECIAL_TOKENS_IDS[-1])
    
    inputs = [torch.stack(list(map(lambda x: torch.from_numpy( \
        np.array(x[index_name])), batch)), dim=0) for index_name in batch[0].keys()]

    inputs = [input_tensor.type(torch.LongTensor) for input_tensor in inputs]

    if use_cuda and torch.cuda.is_available():
        inputs = [input_tensor.cuda() for input_tensor in inputs]   

    return inputs  

The last part is the creation of the DialogDataset objects and the call to the DataLoader.

In [53]:
training_set = DialogDataset(pairs_train) 
validation_set = DialogDataset(pairs_eval)

In [54]:
TRAIN_BATCH_SIZE = 32 
EVAL_BATCH_SIZE = 64 

dataloader_train = DataLoader(training_set, batch_size=TRAIN_BATCH_SIZE, shuffle=False, 
                              collate_fn=custom_collate_fn, num_workers=0) 

dataloader_valid = DataLoader(validation_set, batch_size=EVAL_BATCH_SIZE, shuffle=False,
                              collate_fn=custom_collate_fn, num_workers=0)

The choice of the batch sizes was made carefully as the model along with the batch of the biggest sequence length must be able to fit in RAM memory.

### Training procedure

As we proceed with the training we denifed a few helpful methods. 

In [57]:
def train(model, dataloader, optimizer):
    """
    Handles forward and backward pass for an epoch.
    
    :return : the cumulative loss of the epoch
    """
    epoch_loss = 0.0
    model.train()
    
    for i_batch, (input_ids, attention_mask, category_ids, label_ids) in enumerate(dataloader):
        
        loss, logits = model(input_ids, attention_mask, category_ids, labels=label_ids)

        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()  
        
    return epoch_loss    

In [44]:
def evaluate(model, dataloader):
    """
    Handles forward pass on the validation set
    
    :return : cumulative loss on validation set
    """
    epoch_loss = 0.0
    model.eval()
    with torch.no_grad():
        
        for i_batch, (input_ids, attention_mask, category_ids, label_ids) in enumerate(dataloader):
            
            loss, logits = model(input_ids, attention_mask, category_ids, labels=label_ids)
            epoch_loss += loss.item()
            
    return epoch_loss

The **_train_** and **_evaluate_** functions perform the passes needed for an epoch, while **_early_stopping_** checks the generalization error produced in the validation set and based on some patience parameters it makes decision whether to stop training procudure or not. If it find a new min in validation loss it calls **_checkpoint_state_** which saves the model's weights along other configs into a specified directory.

To mention as a fact only the weights of the model when checkpointed occupied almost 500MB.

In [45]:
def checkpoint_state(model, tokenizer, output_dir=None):
    """
    Function that saves models weights and configuration as well as tokenizer's voc.
    
    :param model: to checkpoint
    :param tokenizer: to checkpoint
    :param output_dir: directory where checkpointed files will be created
    :retun :
    """
    
    if output_dir is None:
        output_dir = 'metalwoz-v1/checkpoint_{}'.format(datetime.now().strftime(TIME_FORMAT))
    
    try:
        os.mkdir(output_dir)
    except FileExistsError:
        pass
    
    output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
    output_config_file = os.path.join(output_dir, CONFIG_NAME)  
    
    torch.save(model.state_dict(), output_model_file) # checkpoint weights
    model.config.to_json_file(output_config_file)     # configuration
    tokenizer.save_vocabulary(output_dir)             # vocabulary

In [46]:
class EarlyStoppingException(Exception):
    """
    Wrapper class to denote a specified exception for Early Stopping
    """
    def __init__(self, message):

        super().__init__(message)

In [47]:
def early_stopping(min_loss, cur_patience, max_patience, epoch_eval_loss):
    """
    Function that checks validation loss and if it has a new min value saves a checkpoint
    otherwise updates the patience and if it is exhausted raise an exception
    """
    if (epoch_eval_loss >= min_loss):  
        cur_patience += 1
        if (cur_patience >= max_patience):
            raise EarlyStoppingException("Execution terminated due to Early Stopping")
    else:
        print("New min validation loss}")
        checkpoint_state(model, tokenizer) # checkpointing
        print("New checkpoint created")
        min_loss, cur_patience = epoch_eval_loss, 0    
        
    return min_loss, cur_patience    

Finally **_training_procedure_** uses all those functions to perform the training.

The optimizer that we used was Adam (with a learning rate of 0.0001) which has proven to be quite faster to converge than vanillia SGD. 

In [48]:
def train_procedure(model, epochs, dataloader_train, dataloader_valid, lr, use_cuda, min_loss, max_patience, cur_patience):
    """
    """
    if use_cuda and torch.cuda.is_available():
        model.cuda()

    optimizer = optim.Adam(model.parameters(), lr=lr) # , weight_decay=0.001 # TODO review those values
    for epoch in range(epochs):

        train_loss = train(model, dataloader_train, optimizer) / len(dataloader_train)
        print("Loss on train set: \t\t epoch {} : {:.4f}".format(epoch, train_loss))

        eval_loss = evaluate(model, dataloader_valid) / len(dataloader_valid)     
        print("Loss on validation set: \t epoch {} : {:.4f}".format(epoch, eval_loss))    

        try:
            min_loss, cur_patience = early_stopping(min_loss, cur_patience, max_patience, eval_loss) 
        except EarlyStoppingException as e:
            print("{} at epoch: {}".format(e, epoch))
            break    

This is a fine-tuning process as we start with a model with pretrained weights. Usually fine-tunig is performed for a small number of epochs. Here we set a maximum of 10 epochs and we monitor validation loss in order to early stop the procedure with a patience of 3 epochs.

The act of training took place in Kaggle which provides access to GPUs accelarators as it was impossible to be handled by a commodity GPU. Even with a strong GPU like Tesla P100 every training epoch needed a bit less than an hour to be finished.

In [51]:
epochs = 10
lr = 1e-4
min_loss, max_patience, cur_patience = np.inf, 3, 0

train_procedure(model, epochs, dataloader_train, dataloader_valid, lr, use_cuda, min_loss, max_patience, cur_patience)

Loss on train set: 		 epoch 0 : 6.6461
Loss on validation set: 	 epoch 0 : 6.3160
New min validation loss}
New checkpoint created
Loss on train set: 		 epoch 1 : 6.1977
Loss on validation set: 	 epoch 1 : 5.8961
New min validation loss}
New checkpoint created
Loss on train set: 		 epoch 2 : 5.9382
Loss on validation set: 	 epoch 2 : 5.8530
New min validation loss}
New checkpoint created
Loss on train set: 		 epoch 3 : 5.8087
Loss on validation set: 	 epoch 3 : 5.8559
Loss on train set: 		 epoch 4 : 5.7147
Loss on validation set: 	 epoch 4 : 5.8270
New min validation loss}
New checkpoint created
Loss on train set: 		 epoch 5 : 5.6415
Loss on validation set: 	 epoch 5 : 5.8311
Loss on train set: 		 epoch 6 : 5.5931
Loss on validation set: 	 epoch 6 : 5.8536
Loss on train set: 		 epoch 7 : 5.5158
Loss on validation set: 	 epoch 7 : 5.8668
Execution terminated due to Early Stopping at epoch: 7


From the above execution results(which is on a sample of the data) and the experiments that we ran on Kaggle we saw that in general the model could not reduce the high values of CrossEntropyLoss. 

We expected the model to reduce the loss on validation set in the first few epochs and then continue reducing training loss (overfitting area). However after some epochs and with different settings tested the loss get stucked around a high value. 

Thus we assume that the inability of the model to learn properly is probably caused by a bug in the developed code.

### Inference

We continue our work with the mechanisms needed in order to interact with the bot. At the core of those mechanisms is the decoding procedure.  

The simpler approach when having the output of the model is to pass them through the softmax activation function. turning them so to probabilities and then pick the word with the highest value. 

However choosing the most probable word or sequences of words as in Beam-search approach doesn't seems to functions well. In fact the language that is produced by those methods is far from that that human produces as it is mentioned in [3]. 

To deal with this issue two new approaches came up in the past year. 

Top-k samples each responded word out of the k most probable tokens based on their probabilities, while the top-p approach samples from the most probable tokens that have a cumulative probability of p. This gives it the ability to focus on the crusial portion of the probability function either this is broad or narrow. 

Those methods along with greedy approach are implemented in the **_decoding_** function.

In [17]:
def format_input(history, tokenizer, reply_so_far):
    """
    Functions that creates the various sequements required by the model as input
    from the history and the reply developed so far.
    
    :return : segments as tensors, no padding/masking is needed as we have single input 
    """
    history = [tokenizer.encode(phrase) for phrase in history]
    
    instance = create_model_inputs(history, reply_so_far, tokenizer, with_eos=False)
    
    input_ids = torch.tensor(instance["input_ids"]).unsqueeze(0)
    token_type_ids = torch.tensor(instance["token_type_ids"]).unsqueeze(0)
    
    return input_ids, token_type_ids

In [3]:
def decoding(probs, logits, method="top_p"):
    """
    Functions that selects the next token to be emmited. Three different approaches are implemented: 
    
    Greedy: the most probable token is selected.
    Top-k : sample out of the k most probable tokens based on their probabilities 
    Top-p : sample from the more probable tokens that have a cumulative probability of p. 
    
    :param logits: 
    :param method: the decoding method to be used, Values={'greedy', 'top_k', 'top_p'}
    :return: the selected token
    """
    top_k = 40 # sample from the 100 most probable tokens based on their probs
    top_p = 0.9 # sample from the n most probable tokens that have a cumulative probability at least 0.9 
    
    if method == "greedy":
        return torch.argmax(probs).item()
    
    elif method == "top_k":        
        prob_k = probs.topk(top_k)[0][-1].item() # value of the 100th most probable
        probs[probs < prob_k] = 0   # cut off the tail  
        
    elif method == "top_p":
        probs_sorted, probs_indexes = probs.sort(dim=-1, descending=True) # start the cumulation from the most probable token in descending order
        cum_probs = probs_sorted.cumsum(dim=-1)
        
        indices = cum_probs > top_p 
        indices[1:] = indices[:-1].clone()
        indices[0] = 0 # at least one token is preserved 
        
        probs[probs_indexes[indices]] = 0
    
    word = torch.multinomial(probs, 1).item()
    # TODO handle the case that special token was emitted in the first pick
    
    return word

Function **_infer_answer_** generates word by word the bot answer, based on user input and previous history. At each iteration history, which includes the so far model emmited words, is passed again into the model and the next token is generated until we find a special token or we reach the predifined max sentence length.  

In [19]:
def infer_answer(history, model, tokenizer, method="top_p"):
    """
    Function that generates word by word the bot answer, based on user input and previous history.
    
    :param history: a list of past sentences and last user's input, in plain text
    :param model: the model to be used for inference
    :return: a list with the words of the answer in plain text 
    """
    model.eval()
    reply_so_far = []
    with torch.no_grad():
    
        for i in range(max_sentence_length):
            
            input_ids, category_ids = format_input(history, tokenizer, reply_so_far)
            outputs = model(input_ids=input_ids, token_type_ids=category_ids)
            logits = outputs[0]
            logits = logits[0, -1, :] / temperature # keep last 
            probs = F.softmax(logits, dim=-1) 
            word = decoding(probs, logits, method=method) 
            
            if word in SPECIAL_TOKENS_IDS: # we stop inference if we find a special token without emitting this token
                break
            reply_so_far.append(word)
            
        answer_text = tokenizer.decode(reply_so_far, skip_special_tokens=True)    
        return answer_text

In the last of our functions we gather user input and we append it in history then we pass it as input in the model and we expect the answer. The answer is appended in the history as well and after that a new round of interaction may start.

We also have the ability to choose between the aforementioned decoding methods. 

In [5]:
def interact_with_bot(model, tokenizer, method='top_p'):
    """
    """
    bot_prompt = "bot:>>> "
    user_prompt = "user:>>> "

    history = []
    print(bot_prompt + "Hello how may I help you?")
    user_input = input(user_prompt)
    
    while user_input != "\q": # TODO check if we need to truncate user input to not exceed max_length
        
        history.append(user_input)
        answer = infer_answer(history, model, tokenizer, method=method)
        history.append(answer)
        
        history = history[-(2*max_history+1):]  # keep the same history as in the training 
        
        print(bot_prompt + answer)
        
        user_input = input(user_prompt) 

Finally we initiate a model with some previously checkpointed weights.

In [15]:
checkpoint_dir = "metalwoz-v1/checkpoint_20191115_1645"

model_loaded ,tokenizer_loaded  = load_checkpoint(checkpoint_dir)

add_special_tokens_(model_loaded, tokenizer_loaded, SPECIAL_TOKENS_DICT)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


New tokens added to model: 5


### Interaction with the bot

As we expected based on what we observed earlier the infered sentences are not good. 

We tried with different decoding approaches. Sampling methods present similar results while the greedy approach produced mainly repetive words.

In [67]:
interact_with_bot(model_loaded, tokenizer_loaded, method='top_p')

bot:>>> Hello how may I help you?
user:>>> can you please tell me what time it is?
bot:>>> will it be on pm, may you be? do you? pm
user:>>> this is not really helpful. could you tell me again?
bot:>>> $ do? august, pm you will in bpjs? what will your help you, what
user:>>> nevermind bye!
bot:>>> the next one. account, / 4 you have a friend? time
user:>>> \q


In [24]:
interact_with_bot(model_loaded, tokenizer_loaded, method='top_k')

bot:>>> Hello how may I help you?
user:>>> do you have any suggestions for a restaurant?
bot:>>> i can go to any restaurant but your name?? it is your number. your date account?
user:>>> just tell me a restaurant
bot:>>> ??.. the
user:>>> this isn't helpful could you repeat?
bot:>>> the day the day you you, or in the microwave and the m -'what are you what
user:>>> \q


In [68]:
interact_with_bot(model_loaded, tokenizer_loaded, method='top_p')

bot:>>> Hello how may I help you?
user:>>> do you have any suggestions for a restaurant?
bot:>>> ' a is a restaurant
user:>>> is that a name?
bot:>>> i am your can be a what do you will, ok ok? how am?'it '
user:>>> ok whatever you say
bot:>>> that was at your lap what are the weather, i
user:>>> \q


In [58]:
interact_with_bot(model_loaded, tokenizer_loaded, method='greedy')

bot:>>> Hello how may I help you?
user:>>> can you propose me a restaurant?
bot:>>> lunch? dinner? dinner? dinner? dinner? dinner? dinner? dinner? dinner? dinner?
user:>>> \q


### References

[1] Transformers Library: https://github.com/huggingface/transformers

[2] Medium Tutorial: https://medium.com/huggingface/how-to-build-a-state-of-the-art-conversational-ai-with-transfer-learning-2d818ac26313

[3] The Curious Case of Neural Text Degeneration: https://arxiv.org/abs/1904.09751 